In [6]:
# Convert PDF to text

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/velissarioschristodoulou/Documents/Github Repos/ModalityFinance/modalitybackend/media/FY23-Q1-Combined-NIKE-Press-Release-Schedules-FINAL_bFLU6ao.pdf") 
pages = loader.load_and_split()

In [7]:
text = ""
for page in pages:
    text += page.page_content
print(text)

Investor Contact: Media Contact:
Paul Trussell KeJuan Wilkins
investor.relations@nike.com media.relations@nike.com
NIKE, INC. REPORTS FISCAL 2023  FIRST QUARTER  RESULTS
BEAVERTON, Ore., Sept. 29, 2022  — NIKE, Inc. (NYSE:NKE) today reported fiscal 2023  financial 
results for its first quarter  ended August 31, 2022 . 
•First quarter  reported revenues were $12.7 billion , up 4 percent  compared to the prior year  and 
up 10 percent  on a currency-neutral basis*
•NIKE Direct sales were $5.1 billion , up 8 percent  on a reported basis and up 14 percent  on a 
currency-neutral basis
•NIKE Brand Digital sales increased 16 percent  on a reported basis, or 23 percent on a 
currency-neutral basis, led by 46 percent growth in EMEA
•Gross margin decreased 220 basis points to 44.3 percent
•Diluted earnings per share for the quarter was $0.93 , down 20 percent
"Our strong start to FY23  highlights the depth and breadth of NIKE’s global portfolio, as we 
continue to manage through volatility,” s

In [9]:
# Create a prompt template for parsing information

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate 

template_string_metrics = """ {report} \n
From the above report, extract the following information in a JSON format: \n
* Company name \n
* Date of report \n
* Headquarters \n
* Industry \n 
* Revenue \n 
* Cost of sales \n
* Gross profit \n 
* Gross margin \n
* Net income
* Cash and equivalents \n 

Then compute the EBITDA and also give me that in the JSON output. 

Then explain how you computed the EBITDA
"""

template_string_tables = """ {report} \n
From the above report, extract all the tables and their title. Return them in a nicely formatted way. 
"""

prompt_template_metrics = ChatPromptTemplate.from_template(template=template_string_metrics)
prompt_template_metrics = ChatPromptTemplate.from_template(template=template_string_tables)

In [10]:
chat = ChatOpenAI(temperature=0.7)

input = prompt_template_metrics.format_messages(report=text)
input_tables = prompt_template_metrics.format_messages(report=text)

/Users/velissarioschristodoulou/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
output = chat(input)
output_tables = chat(input_tables)

/Users/velissarioschristodoulou/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


KeyboardInterrupt: 

In [21]:
print(output.content)

{
  "Company name": "NIKE, Inc.",
  "Date of report": "September 29, 2022",
  "Headquarters": "Beaverton, Oregon",
  "Industry": "Athletic footwear and apparel",
  "Revenue": "$12,687 million",
  "Cost of sales": "$7,072 million",
  "Gross profit": "$5,615 million",
  "Gross margin": "44.3%",
  "Net income": "$1,468 million",
  "Cash and equivalents": "$7,226 million",
  "EBITDA": "$2,120 million"
}

To compute EBITDA, we can use the formula:
EBITDA = Net income + Interest expense + Income tax expense + Depreciation + Amortization

Given that the report does not provide information on Depreciation and Amortization, we can approximate EBITDA by considering it as EBIT (Earnings Before Interest and Taxes), since EBITDA is commonly used as a proxy for operating cash flow. 

Therefore, EBITDA ≈ EBIT = Net income + Interest expense + Income tax expense = $1,468 million + $13 million + $360 million = $1,841 million

So, the computed EBITDA for NIKE, Inc. is approximately $2,120 million.


In [ ]:
print(output_tables.content)

In [13]:
# Convert json object string to a python dictionary
import json

out = json.loads(output.content)